# Imports

In [1]:
import os
import glob
import polars as pl

import datetime
# from datetime import date, time, timedelta, datetime
from datetime import time

import re
import pickle
import json
import httpx

import pandas_market_calendars as mcal

import importlib
import config
importlib.reload(config)

# import time
from time import sleep

from IPython.display import display, Markdown

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CustomJSTickFormatter, DatetimeTickFormatter, DatetimeTicker, HoverTool,CustomJSHover, LinearAxis, Range1d
from bokeh.io import output_notebook
from bokeh.io import output_file, save, show, reset_output

from theta_data_v_3_client import client

from theta_data_v_3_client.api.stock.stock_snapshot_trade import sync as stock_snapshot_trade
from theta_data_v_3_client.api.option.option_list_contracts import sync as option_list_contracts
 

In [2]:
SYMBOL = "AAPL"
START_DATE = datetime.date(2025, 8, 20)
END_DATE = datetime.date(2025, 8, 21)
# TEST_INTERVAL = StockHistoryOhlcInterval.VALUE_13
# TEST_FORMAT_EOD = StockHistoryEodFormat.NDJSON
# TEST_FORMAT_OHLC = StockHistoryOhlcFormat.NDJSON
START_TIME = datetime.time(9, 30)
END_TIME = datetime.time(16, 0)
TEST_TIME="09:30:01.000"
EXPIRATION_DATE = datetime.date(2025, 8, 22)
INDEX_SYMBOL="SPX"

In [3]:
# Example: create an API client instance
base_url = "http://localhost:25504/v3"  # Change to your API's base URL

client1 = client.Client(base_url=base_url)
 
# Example: perform a simple API call (replace with an actual endpoint/method)
try:
    # http://localhost:25503/v3/stock/snapshot/trade?symbol=AAPL
    response = stock_snapshot_trade(client=client1, symbol="AAPL")
    print(f"Client initialized successfully. {response}")
except Exception as e:
    print(f"Error initializing or calling client: {e}")

Client initialized successfully. timestamp,symbol,sequence,size,condition,price
2025-08-27T14:55:14.706,"AAPL",40429619,2,115,230.080



In [4]:
base_url_1 = "http://localhost:25503/v3"  # Change to your API's base URL

client2 = client.Client(base_url=base_url_1)
 
# Example: perform a simple API call (replace with an actual endpoint/method)
try:
    # http://localhost:25504/v3/option/list/contracts?symbol=AAPL
    response = option_list_contracts(client=client2, request_type="trade", symbol="AAPL", date=START_DATE)
    print(f"Client initialized successfully. {response}")
except Exception as e:
    print(f"Error initializing or calling client: {e}")

Client initialized successfully. symbol,expiration,strike,right
"AAPL","2025-08-29",215.000,"PUT"
"AAPL","2025-08-29",215.000,"CALL"
"AAPL","2025-09-12",215.000,"PUT"
"AAPL","2025-09-12",215.000,"CALL"
"AAPL","2025-09-26",215.000,"PUT"
"AAPL","2025-09-26",215.000,"CALL"
"AAPL","2025-11-21",215.000,"CALL"
"AAPL","2025-11-21",215.000,"PUT"
"AAPL","2026-12-18",340.000,"CALL"
"AAPL","2025-12-19",215.000,"CALL"
"AAPL","2025-12-19",215.000,"PUT"
"AAPL","2027-01-15",340.000,"CALL"
"AAPL","2026-01-16",215.000,"CALL"
"AAPL","2026-01-16",215.000,"PUT"
"AAPL","2025-08-29",217.500,"PUT"
"AAPL","2025-08-29",217.500,"CALL"
"AAPL","2026-12-18",210.000,"PUT"
"AAPL","2026-12-18",210.000,"CALL"
"AAPL","2027-01-15",210.000,"CALL"
"AAPL","2027-01-15",210.000,"PUT"
"AAPL","2026-01-16",85.000,"PUT"
"AAPL","2027-12-17",205.000,"CALL"
"AAPL","2027-12-17",205.000,"PUT"
"AAPL","2026-12-18",80.000,"CALL"
"AAPL","2027-01-15",80.000,"PUT"
"AAPL","2025-08-29",220.000,"CALL"
"AAPL","2025-08-29",220.000,"PUT"
"AAPL",

# Function Defintions

In [5]:
# Extract the date from a filename where the date is in the format YYYYMMDD convert to datetime
def extract_date_from_filename_datetime(filename):
    date = re.search(r'\d{8}', filename)
    if date:
        return datetime.strptime(date.group(0), '%Y%m%d')
    return None

# Convert an integer representing a time in the format HHMMSS to a time object
def int_to_time(time_int):
    time_str = str(time_int).zfill(6)
    hour = int(time_str[:2])
    minute = int(time_str[2:4])
    second = int(time_str[4:])
    return time(hour=hour, minute=minute, second=second)

def event_time_to_burst_filename(event_dt):
    return f"Burst_{event_dt.strftime('%Y%m%d_%H%M%S')}.csv"

def get_symbol_burst_data(event_dt):
    filename = event_time_to_burst_filename(event_dt)
    filepath = os.path.join("./Data_Burst_Symbols", filename)
    # print(f"Looking for file: {filepath}")
    if os.path.exists(filepath):
        df = pl.read_csv(filepath, has_header=True)
        df = df.rename({
            df.columns[0]: "Symbol",
            df.columns[1]: "NumTrades",
            df.columns[2]: "NumQuotes",
            df.columns[3]: "BBO"
        })
        # Validate and clean the Symbol column
        df = df.with_columns([
            pl.col("Symbol").fill_null("").cast(pl.String)  # Replace nulls and cast to string
        ])
        # Convert rows to Polars-compatible List(Struct)
        return [
            {"Symbol": row["Symbol"], "NumTrades": row["NumTrades"], "NumQuotes": row["NumQuotes"], "BBO": row["BBO"]}
            for row in df.to_dicts()
        ]
    else:
        # Return a Polars-compatible List(Struct) with a single default value
        return [
            {"Symbol": "", "NumTrades": 0, "NumQuotes": 0, "BBO": 0}
        ]

# Initialize Parameters

In [6]:

print(f"Current working directory: {os.getcwd()}")

today_str = datetime.datetime.today().strftime('%Y%m%d')
print(f"Today's date string: {today_str}")

# make today-str year/month/day format
today_str1 = datetime.datetime.today().strftime('%Y/%m/')
print(f"Today's date string in year/month/day format: {today_str1}{today_str}")

csv_path = f"./Code/Data/{today_str}_hft1.csv"
csv_path_2 = f"./Data/{today_str}_hft2.csv"
csv_path_3 = f"./Data/{today_str}_hft3.csv"
burst_path = f"./Data_Burst/{today_str}.csv"
symbol_path = glob.glob(f"./Data_Burst_Symbols/Burst_{today_str}_??????.csv")
print (f"Symbol files: {symbol_path}")
print(f"Today's csv_path: {csv_path}")
parquet_path = f"./Processed_Data/{today_str1}{today_str}_hft1.parquet"
burst_detail_parquet_path = f"./Processed_Data/{today_str1}{today_str}_burst_detail.parquet"
print(f"Today's parquet_path: {parquet_path}")
last_size_path = f"./Code/Data/{today_str}_hft1_lastsize.pkl"

Current working directory: c:\Users\vande\Desktop\Projects\Ki2_Alerts\Code
Today's date string: 20250827
Today's date string in year/month/day format: 2025/08/20250827
Symbol files: []
Today's csv_path: ./Code/Data/20250827_hft1.csv
Today's parquet_path: ./Processed_Data/2025/08/20250827_hft1.parquet


In [7]:
burst_symbol_files = glob.glob(f"./Data_Burst_Symbols/Burst_????????_??????.csv")
data_burst_files = glob.glob(f"./Data_Burst/????????.csv")
print(f'Found {len(burst_symbol_files)} burst symbol files')
print(f'Found {len(data_burst_files)} data burst files')
data_burst_files


Found 4805 burst symbol files
Found 21 data burst files


['./Data_Burst\\20250722.csv',
 './Data_Burst\\20250723.csv',
 './Data_Burst\\20250724.csv',
 './Data_Burst\\20250725.csv',
 './Data_Burst\\20250728.csv',
 './Data_Burst\\20250729.csv',
 './Data_Burst\\20250730.csv',
 './Data_Burst\\20250731.csv',
 './Data_Burst\\20250801.csv',
 './Data_Burst\\20250804.csv',
 './Data_Burst\\20250805.csv',
 './Data_Burst\\20250806.csv',
 './Data_Burst\\20250807.csv',
 './Data_Burst\\20250808.csv',
 './Data_Burst\\20250811.csv',
 './Data_Burst\\20250812.csv',
 './Data_Burst\\20250813.csv',
 './Data_Burst\\20250815.csv',
 './Data_Burst\\20250819.csv',
 './Data_Burst\\20250820.csv',
 './Data_Burst\\20250821.csv']

In [ ]:
import re
from datetime import datetime

def event_time_to_filename(event_dt):
    """
    Convert an event datetime to a filename format.
    Assumes event_dt is a datetime object.
    """
    return f"Burst_{event_dt.strftime('%Y%m%d_%H%M%S')}.csv"

def get_symbol_burst_data(event_dt):
    filename = event_time_to_filename(event_dt)
    filepath = os.path.join("./Data_Burst_Symbols", filename)
    # print(f"Looking for file: {filepath}")
    if os.path.exists(filepath):
        df = pl.read_csv(filepath, has_header=True)
        df = df.rename({
            df.columns[0]: "Symbol",
            df.columns[1]: "NumTrades",
            df.columns[2]: "NumQuotes",
            df.columns[3]: "BBO"
        })
        # Validate and clean the Symbol column
        df = df.with_columns([
            pl.col("Symbol").fill_null("").cast(pl.String)  # Replace nulls and cast to string
        ])
        # Convert rows to Polars-compatible List(Struct)
        return [
            {"Symbol": row["Symbol"], "NumTrades": row["NumTrades"], "NumQuotes": row["NumQuotes"], "BBO": row["BBO"]}
            for row in df.to_dicts()
        ]
    else:
        # Return a Polars-compatible List(Struct) with a single default value
        return [{"Symbol": "", "NumTrades": 0, "NumQuotes": 0, "BBO": 0}]

# Define the struct for the symbol burst data
symbol_struct = pl.Struct([
    pl.Field("Symbol", pl.String),
    pl.Field("NumTrades", pl.Int64),
    pl.Field("NumQuotes", pl.Int64),
    pl.Field("BBO", pl.Int64)
])
    
#Read a Burst_Data file and convert to Polars Dataframe
test_df = pl.read_csv(data_burst_files[0], has_header=False)
test_df = test_df.rename({
    'column_1': 'Event_Date_Time',
    'column_2': 'Grade',
    'column_3': 'Prog_Type',
    'column_4': 'Bckgrnd',
    'column_5': 'S1',
    'column_6': 'S2',
    'column_7': 'CT',
    'column_8': 'CS',
    'column_9': 'CB'
})
#Extract Date from file name
file_name = data_burst_files[0].split('/')[-1]
print(f'Processing File: {file_name}')

# Extract the date part (YYYYMMDD) from the filename
date_match = re.search(r'(\d{8})', file_name)
if date_match:
    date_str = date_match.group(1)
    date_obj = datetime.strptime(date_str, "%Y%m%d")
    print(f'Extracted Date: {date_str}')
    print(f'Date as datetime: {date_obj}')
else:
    print("No date found in filename.")
print(f'Extracted Date: {date_str}')

test_df = test_df.with_columns(pl.col('Event_Date_Time').map_elements(int_to_time, return_dtype=pl.Time).alias('Event_Date_Time'))
test_df = test_df.with_columns(pl.col('Event_Date_Time').map_elements(lambda x: datetime.combine(date_obj, x), return_dtype=pl.Datetime).alias('Event_Date_Time'))

# Apply the function to the DataFrame
test_df = test_df.with_columns([
    pl.col("Event_Date_Time").map_elements(
        get_symbol_burst_data,
        return_dtype=pl.List(symbol_struct)
    ).alias("Burst_Symbols")
])

# Add a new column with ms_of_day based on Event_Date_Time
def get_ms_of_day(dt):
    return (dt.hour * 3600 + dt.minute * 60 + dt.second) * 1000 + int(dt.microsecond / 1000)

test_df = test_df.with_columns(
    pl.col("Event_Date_Time").map_elements(get_ms_of_day, return_dtype=pl.Int64).alias("ms_of_day")
)

# http://127.0.0.1:25510/v2/hist/stock/trade?root=AAPL&start_date=20240102&end_date=20240102

# http://127.0.0.1:25510/v2/hist/stock/ohlc?root=AAPL&start_date=20240102&end_date=20240102&ivl=60000

# http://127.0.0.1:25510/v2/hist/stock/quote?root=AAPL&start_date=20240102&end_date=20240102&ivl=60000


# save parquet file to processed data directory
# year = date_obj.strftime("%Y")
# month = date_obj.strftime("%m")
# output_dir = f"./Processed_Data/{year}/{month}"
# os.makedirs(output_dir, exist_ok=True)
# output_path = f"{output_dir}/{date_str}_burst_symbols.parquet"
# test_df.write_parquet(output_path)
test_df

Processing File: Data_Burst\20250722.csv
Extracted Date: 20250722
Date as datetime: 2025-07-22 00:00:00
Extracted Date: 20250722


Event_Date_Time,Grade,Prog_Type,Bckgrnd,S1,S2,CT,CS,CB,Burst_Symbols,ms_of_day
datetime[μs],i64,i64,i64,i64,i64,i64,i64,i64,list[struct[4]],i64
2025-07-22 09:30:01,2,0,0,149,296,1,0,0,"[{""AAPD"",0,188,49}, {""AAPL"",559,881,221}, … {""URTY"",6,865,143}]",34201000
2025-07-22 09:30:11,3,0,0,105,172,5,1,1,"[{""AAPB"",1,457,105}, {""AAPD"",0,454,97}, … {""YINN"",120,524,196}]",34211000
2025-07-22 09:30:14,3,0,0,79,107,6,2,1,"[{""AMD"",37,181,75}, {""AMDD"",0,998,181}, … {""ZETA"",53,302,73}]",34214000
2025-07-22 09:30:31,3,1,0,89,222,7,2,2,"[{""AAPB"",2,263,47}, {""AAPD"",0,375,127}, … {""XRT"",88,480,133}]",34231000
2025-07-22 09:31:05,3,1,0,84,216,8,2,3,"[{""AAPB"",6,214,36}, {""AAPD"",0,258,78}, … {""XRT"",20,649,181}]",34265000
…,…,…,…,…,…,…,…,…,…,…
2025-07-22 15:59:20,3,0,0,79,273,366,213,150,"[{""AAP"",108,294,87}, {""AAPL"",113,498,154}, … {""ZTO"",90,346,81}]",57560000
2025-07-22 15:59:56,3,0,0,138,292,367,214,150,"[{""AAL"",33,83,12}, {""AAPB"",2,303,87}, … {""YINN"",15,266,70}]",57596000
2025-07-22 15:59:58,3,1,0,76,247,368,214,151,"[{""AAPL"",108,502,174}, {""AAPU"",0,232,49}, … {""ZGN"",113,326,65}]",57598000


In [ ]:
def process_burst_file(file_path):
    #Read a Burst_Data file and convert to Polars Dataframe
    df = pl.read_csv(file_path, has_header=False)
    df = df.rename({
        'column_1': 'Event_Date_Time',
        'column_2': 'Grade',
        'column_3': 'Prog_Type',
        'column_4': 'Bckgrnd',
        'column_5': 'S1',
        'column_6': 'S2',
        'column_7': 'CT',
        'column_8': 'CS',
        'column_9': 'CB'
    })
    #Extract Date from file name
    file_name = file_path.split('/')[-1]
    print(f'Processing File: {file_name}')

    # Extract the date part (YYYYMMDD) from the filename
    date_match = re.search(r'(\d{8})', file_name)
    if date_match:
        date_str = date_match.group(1)
        date_obj = datetime.strptime(date_str, "%Y%m%d")
        print(f'Extracted Date: {date_str}')
        print(f'Date as datetime: {date_obj}')
    else:
        print("No date found in filename.")
        return None
    print(f'Extracted Date: {date_str}')

    df = df.with_columns(pl.col('Event_Date_Time').map_elements(int_to_time, return_dtype=pl.Time).alias('Event_Date_Time'))
    df = df.with_columns(pl.col('Event_Date_Time').map_elements(lambda x: datetime.combine(date_obj, x), return_dtype=pl.Datetime).alias('Event_Date_Time'))

    # Apply the function to the DataFrame
    df = df.with_columns([
        pl.col("Event_Date_Time").map_elements(
            get_symbol_burst_data,
            return_dtype=pl.List(symbol_struct)
        ).alias("Burst_Symbols")
    ])

    # Add a new column with ms_of_day based on Event_Date_Time
    def get_ms_of_day(dt):
        return (dt.hour * 3600 + dt.minute * 60 + dt.second) * 1000 + int(dt.microsecond / 1000)

    df = df.with_columns(
        pl.col("Event_Date_Time").map_elements(get_ms_of_day, return_dtype=pl.Int64).alias("ms_of_day")
    )

    return df

# Processing Chain Description

1) Call process_burst_file - This is looking for a single file path from the Data_Burst Folder. This will generate a dataframe for the basic information for the burst that appears in the center pane of the Burst_Monitor,  I Nthis analysis this is called test_df
2) The next function in the chain is fetch_trade_history_for_burst(test_df) .  This will capture data from the Thetadata stock/history/trade , stock/history/quote and stock/history/ohlc endpoints for a time perios on second before the burst through one second after the burst.  OHLS data will be on a 1 second interval, but this can be reducted to as low as 10ms.  This goes through each row individually and process the burst_symbols it finds and expand the data frame to include one row per symbol with the same Event_Date_Time.  Columns containing Struct are added called Trade_History, Quote_History and OHLC.  In the analysis that follows this is called Burst_df
3) The next function is refactor_burst_df.  This takes all of the columns generated in the previous step and builds them into a single Combined_Struct Colum, preparing ofr storage.  These collapses the information back to one rwo per Event_Date_Time.  The data fram that reseluts from this is called final_df
4) The next step is to add the original data from test_df back in with the merge ```merged_df = test_df.join(final_df, on="Event_Date_Time", how="left")```
5) This is now saved to parquet in the Processed_Data Folder in a yyyy/mm subfolder with name yyymmdd_burst_detail.parquet with the support of extract_date_strings that generates the necessary string by looking at the analysis date in merged_df
6) The yyymmdd_burst_detail,parquet file can be read back and processed with expand_combined_struct to expand it back into a dataframe that has one row per symbol, the basic brust information provided in the center panel of burst monitor repeated once per symbol roew and columns with structs for Trade_History, Quote_History and OHLC.  In the analysis below this is called original_df.

In [14]:
def process_burst_file(file_path):

    # Define the struct for the symbol burst data
    symbol_struct = pl.Struct([
        pl.Field("Symbol", pl.String),
        pl.Field("NumTrades", pl.Int64),
        pl.Field("NumQuotes", pl.Int64),
        pl.Field("BBO", pl.Int64)
    ])
    
    #Read a Burst_Data file and convert to Polars Dataframe
    test_df = pl.read_csv(file_path, has_header=False)
    test_df = test_df.rename({
        'column_1': 'Event_Date_Time',
        'column_2': 'Grade',
        'column_3': 'Prog_Type',
        'column_4': 'Bckgrnd',
        'column_5': 'S1',
        'column_6': 'S2',
        'column_7': 'CT',
        'column_8': 'CS',
        'column_9': 'CB'
    })
    #Extract Date from file name
    file_name = file_path.split('/')[-1]
    print(f'Processing File: {file_name}')

    # Extract the date part (YYYYMMDD) from the filename
    date_match = re.search(r'(\d{8})', file_name)
    if date_match:
        date_str = date_match.group(1)
        date_obj = datetime.strptime(date_str, "%Y%m%d")
        print(f'Extracted Date: {date_str}')
        print(f'Date as datetime: {date_obj}')
    else:
        print("No date found in filename.")
    print(f'Extracted Date: {date_str}')

    test_df = test_df.with_columns(pl.col('Event_Date_Time').map_elements(int_to_time, return_dtype=pl.Time).alias('Event_Date_Time'))
    test_df = test_df.with_columns(pl.col('Event_Date_Time').map_elements(lambda x: datetime.combine(date_obj, x), return_dtype=pl.Datetime).alias('Event_Date_Time'))

    # Apply the function to the DataFrame
    test_df = test_df.with_columns([
        pl.col("Event_Date_Time").map_elements(
            get_symbol_burst_data,
            return_dtype=pl.List(symbol_struct)
        ).alias("Burst_Symbols")
    ])

    # Add a new column with ms_of_day based on Event_Date_Time
    def get_ms_of_day(dt):
        return (dt.hour * 3600 + dt.minute * 60 + dt.second) * 1000 + int(dt.microsecond / 1000)

    test_df = test_df.with_columns(
        pl.col("Event_Date_Time").map_elements(get_ms_of_day, return_dtype=pl.Int64).alias("ms_of_day")
    )

    # http://127.0.0.1:25510/v2/hist/stock/trade?root=AAPL&start_date=20240102&end_date=20240102

    # http://127.0.0.1:25510/v2/hist/stock/ohlc?root=AAPL&start_date=20240102&end_date=20240102&ivl=60000

    # http://127.0.0.1:25510/v2/hist/stock/quote?root=AAPL&start_date=20240102&end_date=20240102&ivl=60000


    # save parquet file to processed data directory
    # year = date_obj.strftime("%Y")
    # month = date_obj.strftime("%m")
    # output_dir = f"./Processed_Data/{year}/{month}"
    # os.makedirs(output_dir, exist_ok=True)
    # output_path = f"{output_dir}/{date_str}_burst_symbols.parquet"
    # test_df.write_parquet(output_path)
    return test_df

file_path = data_burst_files[0]

test_df=process_burst_file(file_path)
test_df

Processing File: Data_Burst\20250722.csv
Extracted Date: 20250722
Date as datetime: 2025-07-22 00:00:00
Extracted Date: 20250722


Event_Date_Time,Grade,Prog_Type,Bckgrnd,S1,S2,CT,CS,CB,Burst_Symbols,ms_of_day
datetime[μs],i64,i64,i64,i64,i64,i64,i64,i64,list[struct[4]],i64
2025-07-22 09:30:01,2,0,0,149,296,1,0,0,"[{""AAPD"",0,188,49}, {""AAPL"",559,881,221}, … {""URTY"",6,865,143}]",34201000
2025-07-22 09:30:11,3,0,0,105,172,5,1,1,"[{""AAPB"",1,457,105}, {""AAPD"",0,454,97}, … {""YINN"",120,524,196}]",34211000
2025-07-22 09:30:14,3,0,0,79,107,6,2,1,"[{""AMD"",37,181,75}, {""AMDD"",0,998,181}, … {""ZETA"",53,302,73}]",34214000
2025-07-22 09:30:31,3,1,0,89,222,7,2,2,"[{""AAPB"",2,263,47}, {""AAPD"",0,375,127}, … {""XRT"",88,480,133}]",34231000
2025-07-22 09:31:05,3,1,0,84,216,8,2,3,"[{""AAPB"",6,214,36}, {""AAPD"",0,258,78}, … {""XRT"",20,649,181}]",34265000
…,…,…,…,…,…,…,…,…,…,…
2025-07-22 15:59:20,3,0,0,79,273,366,213,150,"[{""AAP"",108,294,87}, {""AAPL"",113,498,154}, … {""ZTO"",90,346,81}]",57560000
2025-07-22 15:59:56,3,0,0,138,292,367,214,150,"[{""AAL"",33,83,12}, {""AAPB"",2,303,87}, … {""YINN"",15,266,70}]",57596000
2025-07-22 15:59:58,3,1,0,76,247,368,214,151,"[{""AAPL"",108,502,174}, {""AAPU"",0,232,49}, … {""ZGN"",113,326,65}]",57598000


# EXternal Data processing

In [ ]:
def fetch_trade_history_for_burst(test_df):
    """
    For each row in test_df, extract Event_Date_Time and Burst_Symbols,
    and fetch trade history for each symbol using thetadata API.
    Returns a Polars DataFrame with the original burst data plus the trade history fields.
    """
    BASE_URL = "http://localhost:25504/v3/stock/history/trade"
    BASE_URL_1 = "http://localhost:25504/v3/stock/history/quote"
    BASE_URL_2 = "http://localhost:25504/v3/stock/history/ohlc"

    FORMAT = "ndjson"

    results = []

    for row in test_df.iter_rows(named=True):
        event_dt = row["Event_Date_Time"]
        burst_symbols = row["Burst_Symbols"]
        if not burst_symbols:
            continue

        # Prepare date
        date_str = event_dt.strftime("%Y%m%d")
        ms_of_day = (event_dt.hour * 3600 + event_dt.minute * 60 + event_dt.second) * 1000 + int(event_dt.microsecond / 1000)
        # Calculate start and end times in ms
        start_ms = max(ms_of_day - 1000, 0)
        end_ms = ms_of_day + 1000

        # Convert ms_of_day to HH.MM.SS.mmm format
        def ms_to_time_str(ms):
            hours = ms // 3600000
            ms %= 3600000
            minutes = ms // 60000
            ms %= 60000
            seconds = ms // 1000
            milliseconds = ms % 1000
            return f"{hours:02d}:{minutes:02d}:{seconds:02d}.{milliseconds:03d}"

        start_time_str = ms_to_time_str(start_ms)
        end_time_str = ms_to_time_str(end_ms)

        for symbol_info in burst_symbols:
            symbol = symbol_info.get("Symbol")
            print(f'Processing Symbol: {symbol} from Event Time: {event_dt}')
            if not symbol:
                continue

            params = {
                "symbol": symbol,
                "date": date_str,
                "start_time": start_time_str,
                "end_time": end_time_str,
                "format": FORMAT
            }

            try:
                response = httpx.get(BASE_URL, params=params, timeout=10)
                response.raise_for_status()
                # Parse NDJSON response (one JSON object per line)
                # trades = [httpx.Response.json(line) for line in response.text.strip().split('\n') if line.strip()]
                trades = [json.loads(line) for line in response.text.strip().split('\n') if line.strip()]
            except Exception as e:
                trades = []

            try:
                response = httpx.get(BASE_URL_1,params=params, timeout=10)
                response.raise_for_status()
                # Parse NDJSON response (one JSON object per line)
                # trades = [httpx.Response.json(line) for line in response.text.strip().split('\n') if line.strip()]
                quotes = [json.loads(line) for line in response.text.strip().split('\n') if line.strip()]
            except Exception as e:
                quotes = []

            try:
                response = httpx.get(BASE_URL_2,params=params, timeout=10)
                response.raise_for_status()
                # Parse NDJSON response (one JSON object per line)
                # trades = [httpx.Response.json(line) for line in response.text.strip().split('\n') if line.strip()]
                ohlc = [json.loads(line) for line in response.text.strip().split('\n') if line.strip()]
            except Exception as e:
                ohlc = []

            # Combine the burst symbol info and trade data
            results.append({
                "Event_Date_Time": event_dt,
                "Symbol": symbol,
                "Burst_Info": symbol_info,
                "Trade_History": trades,
                "Quote_History": quotes,
                "OHLC": ohlc
            })
    # Convert results to Polars DataFrame
    burst_df = pl.DataFrame(results)
    return burst_df

# Example usage:
burst_df = fetch_trade_history_for_burst(test_df)
burst_df

Processing Symbol: AALG from Event Time: 2025-07-24 09:30:01
Processing Symbol: AAPL from Event Time: 2025-07-24 09:30:01
Processing Symbol: AAPU from Event Time: 2025-07-24 09:30:01
Processing Symbol: ACMR from Event Time: 2025-07-24 09:30:01
Processing Symbol: ACWI from Event Time: 2025-07-24 09:30:01
Processing Symbol: ACWX from Event Time: 2025-07-24 09:30:01
Processing Symbol: ADT from Event Time: 2025-07-24 09:30:01
Processing Symbol: AG from Event Time: 2025-07-24 09:30:01
Processing Symbol: AGH from Event Time: 2025-07-24 09:30:01
Processing Symbol: AGNC from Event Time: 2025-07-24 09:30:01
Processing Symbol: AIRR from Event Time: 2025-07-24 09:30:01
Processing Symbol: ALV from Event Time: 2025-07-24 09:30:01
Processing Symbol: AMD from Event Time: 2025-07-24 09:30:01
Processing Symbol: AMDD from Event Time: 2025-07-24 09:30:01
Processing Symbol: AMDG from Event Time: 2025-07-24 09:30:01
Processing Symbol: AMDL from Event Time: 2025-07-24 09:30:01
Processing Symbol: AMUU from E

Event_Date_Time,Symbol,Burst_Info,Trade_History,Quote_History,OHLC
datetime[μs],str,struct[4],list[struct[10]],list[struct[9]],list[struct[8]]
2025-07-24 09:30:01,"""AALG""","{""AALG"",1,204,61}","[{3303805,115,10,12.18,95,32,115,65,255,""2025-07-24T09:30:00.587""}, {3317786,62,675,12.13,62,32,45,1,255,""2025-07-24T09:30:00.822""}, {3317787,66,675,12.13,255,32,66,1,255,""2025-07-24T09:30:00.822""}]","[{5,2,1,0,1,12.3,12.23,0,""2025-07-24T09:30:00""}, {5,1,1,0,1,12.2,12.14,0,""2025-07-24T09:30:01""}, {5,1,1,0,1,12.18,12.12,0,""2025-07-24T09:30:02""}]","[{685,12.13,12.13,12.13,2,12.13,12.13,""2025-07-24T09:30:00""}, {0,0.0,0.0,12.13,0,0.0,0.0,""2025-07-24T09:30:01""}]"
2025-07-24 09:30:01,"""AAPL""","{""AAPL"",144,169,40}","[{3277779,115,17,213.82,95,32,115,60,255,""2025-07-24T09:30:00.016""}, {3277780,115,10,213.82,95,32,115,65,255,""2025-07-24T09:30:00.016""}, … {3405711,96,1,213.92,96,32,115,57,255,""2025-07-24T09:30:02""}]","[{63,1,1,0,1,214.0,213.77,0,""2025-07-24T09:30:00""}, {1,1,65,0,7,213.8,213.74,0,""2025-07-24T09:30:01""}, {1,2,1,0,1,213.98,213.91,0,""2025-07-24T09:30:02""}]","[{6681,213.98,213.77,213.84,169,213.77,213.9,""2025-07-24T09:30:00""}, {506509,213.98,213.69,213.91,844,213.98,213.79,""2025-07-24T09:30:01""}]"
2025-07-24 09:30:01,"""AAPU""","{""AAPU"",4,192,41}","[{3295621,1,1,23.64,255,32,115,1,1,""2025-07-24T09:30:00.418""}, {3295622,1,29,23.64,255,32,115,1,1,""2025-07-24T09:30:00.418""}, … {3395101,115,1,23.67,255,32,115,58,255,""2025-07-24T09:30:01.857""}]","[{13,13,1,0,1,23.7,23.62,0,""2025-07-24T09:30:00""}, {11,11,65,0,65,23.65,23.63,0,""2025-07-24T09:30:01""}, {11,12,65,0,11,23.69,23.66,0,""2025-07-24T09:30:02""}]","[{36,0.0,0.0,0.0,4,0.0,0.0,""2025-07-24T09:30:00""}, {38113,23.67,23.63,23.65,33,23.67,23.66,""2025-07-24T09:30:01""}]"
2025-07-24 09:30:01,"""ACMR""","{""ACMR"",21,166,25}","[{3281324,62,7935,30.26,62,32,45,1,255,""2025-07-24T09:30:00.094""}, {3281349,66,7935,30.26,255,32,66,1,255,""2025-07-24T09:30:00.094""}, … {3404322,115,1,30.26,255,32,115,57,255,""2025-07-24T09:30:01.975""}]","[{1,1,65,0,7,30.4,29.95,0,""2025-07-24T09:30:00""}, {1,4,1,0,7,30.29,30.0,0,""2025-07-24T09:30:01""}, {1,1,1,0,1,30.29,30.0,0,""2025-07-24T09:30:02""}]","[{8307,30.26,30.125,30.25,25,30.125,30.26,""2025-07-24T09:30:00""}, {95,0.0,0.0,30.25,8,0.0,0.0,""2025-07-24T09:30:01""}]"
2025-07-24 09:30:01,"""ACWI""","{""ACWI"",15,151,22}","[{3295182,1,5,131.79,95,32,115,1,1,""2025-07-24T09:30:00.41""}, {3295183,1,123,131.8,95,32,255,1,1,""2025-07-24T09:30:00.41""}, … {3395187,96,1,131.8,96,32,115,57,255,""2025-07-24T09:30:01.858""}]","[{11,11,1,0,7,131.8,131.64,0,""2025-07-24T09:30:00""}, {15,7,1,0,1,131.8,131.66,0,""2025-07-24T09:30:01""}, {15,4,1,0,7,131.8,131.66,0,""2025-07-24T09:30:02""}]","[{9064,131.8,131.8,131.8,16,131.8,131.8,""2025-07-24T09:30:00""}, {511,131.73,131.73,131.79,20,131.73,131.73,""2025-07-24T09:30:01""}]"
…,…,…,…,…,…
2025-07-24 09:30:05,"""XLE""","{""XLE"",26,237,29}","[{572555,115,3,86.41,255,255,115,57,255,""2025-07-24T09:30:04.216""}, {572641,0,200,86.425,255,32,255,57,255,""2025-07-24T09:30:04.247""}, … {577693,96,1,86.58,96,255,115,57,255,""2025-07-24T09:30:05.528""}]","[{10,3,7,0,68,86.45,86.39,0,""2025-07-24T09:30:04""}, {3,2,1,0,7,86.39,86.36,0,""2025-07-24T09:30:05""}, {14,4,7,0,68,86.36,86.34,0,""2025-07-24T09:30:06""}]","[{1799,86.425,86.37,86.39,26,86.37,86.425,""2025-07-24T09:30:04""}, {3471,86.37,86.34,86.37,54,86.34,86.37,""2025-07-24T09:30:05""}]"
2025-07-24 09:30:05,"""XLK""","{""XLK"",63,368,68}","[{571846,115,14,260.87,95,255,115,1,255,""2025-07-24T09:30:04.037""}, {571848,115,20,260.94,95,255,115,73,255,""2025-07-24T09:30:04.037""}, … {578191,115,1,260.7201,255,255,115,57,255,""2025-07-24T09:30:05.694""}]","[{2,3,68,0,68,260.94,260.74,0,""2025-07-24T09:30:04""}, {1,1,7,0,60,260.71,260.7,0,""2025-07-24T09:30:05""}, {1,2,60,0,7,260.78,260.66,0,""2025-07-24T09:30:06""}]","[{4272,260.84,260.71,260.77,64,260.71,260.84,""2025-07-24T09:30:04""}, {131,260.71,260.71,260.77,5,260.71,260.7

In [74]:
burst_df

Event_Date_Time,Symbol,Burst_Info,Trade_History,Quote_History,OHLC
datetime[μs],str,struct[4],list[struct[10]],list[struct[9]],list[struct[8]]
2025-07-24 09:30:01,"""AALG""","{""AALG"",1,204,61}","[{3303805,115,10,12.18,95,32,115,65,255,""2025-07-24T09:30:00.587""}, {3317786,62,675,12.13,62,32,45,1,255,""2025-07-24T09:30:00.822""}, {3317787,66,675,12.13,255,32,66,1,255,""2025-07-24T09:30:00.822""}]","[{5,2,1,0,1,12.3,12.23,0,""2025-07-24T09:30:00""}, {5,1,1,0,1,12.2,12.14,0,""2025-07-24T09:30:01""}, {5,1,1,0,1,12.18,12.12,0,""2025-07-24T09:30:02""}]","[{685,12.13,12.13,12.13,2,12.13,12.13,""2025-07-24T09:30:00""}, {0,0.0,0.0,12.13,0,0.0,0.0,""2025-07-24T09:30:01""}]"
2025-07-24 09:30:01,"""AAPL""","{""AAPL"",144,169,40}","[{3277779,115,17,213.82,95,32,115,60,255,""2025-07-24T09:30:00.016""}, {3277780,115,10,213.82,95,32,115,65,255,""2025-07-24T09:30:00.016""}, … {3405711,96,1,213.92,96,32,115,57,255,""2025-07-24T09:30:02""}]","[{63,1,1,0,1,214.0,213.77,0,""2025-07-24T09:30:00""}, {1,1,65,0,7,213.8,213.74,0,""2025-07-24T09:30:01""}, {1,2,1,0,1,213.98,213.91,0,""2025-07-24T09:30:02""}]","[{6681,213.98,213.77,213.84,169,213.77,213.9,""2025-07-24T09:30:00""}, {506509,213.98,213.69,213.91,844,213.98,213.79,""2025-07-24T09:30:01""}]"
2025-07-24 09:30:01,"""AAPU""","{""AAPU"",4,192,41}","[{3295621,1,1,23.64,255,32,115,1,1,""2025-07-24T09:30:00.418""}, {3295622,1,29,23.64,255,32,115,1,1,""2025-07-24T09:30:00.418""}, … {3395101,115,1,23.67,255,32,115,58,255,""2025-07-24T09:30:01.857""}]","[{13,13,1,0,1,23.7,23.62,0,""2025-07-24T09:30:00""}, {11,11,65,0,65,23.65,23.63,0,""2025-07-24T09:30:01""}, {11,12,65,0,11,23.69,23.66,0,""2025-07-24T09:30:02""}]","[{36,0.0,0.0,0.0,4,0.0,0.0,""2025-07-24T09:30:00""}, {38113,23.67,23.63,23.65,33,23.67,23.66,""2025-07-24T09:30:01""}]"
2025-07-24 09:30:01,"""ACMR""","{""ACMR"",21,166,25}","[{3281324,62,7935,30.26,62,32,45,1,255,""2025-07-24T09:30:00.094""}, {3281349,66,7935,30.26,255,32,66,1,255,""2025-07-24T09:30:00.094""}, … {3404322,115,1,30.26,255,32,115,57,255,""2025-07-24T09:30:01.975""}]","[{1,1,65,0,7,30.4,29.95,0,""2025-07-24T09:30:00""}, {1,4,1,0,7,30.29,30.0,0,""2025-07-24T09:30:01""}, {1,1,1,0,1,30.29,30.0,0,""2025-07-24T09:30:02""}]","[{8307,30.26,30.125,30.25,25,30.125,30.26,""2025-07-24T09:30:00""}, {95,0.0,0.0,30.25,8,0.0,0.0,""2025-07-24T09:30:01""}]"
2025-07-24 09:30:01,"""ACWI""","{""ACWI"",15,151,22}","[{3295182,1,5,131.79,95,32,115,1,1,""2025-07-24T09:30:00.41""}, {3295183,1,123,131.8,95,32,255,1,1,""2025-07-24T09:30:00.41""}, … {3395187,96,1,131.8,96,32,115,57,255,""2025-07-24T09:30:01.858""}]","[{11,11,1,0,7,131.8,131.64,0,""2025-07-24T09:30:00""}, {15,7,1,0,1,131.8,131.66,0,""2025-07-24T09:30:01""}, {15,4,1,0,7,131.8,131.66,0,""2025-07-24T09:30:02""}]","[{9064,131.8,131.8,131.8,16,131.8,131.8,""2025-07-24T09:30:00""}, {511,131.73,131.73,131.79,20,131.73,131.73,""2025-07-24T09:30:01""}]"
…,…,…,…,…,…
2025-07-24 09:30:05,"""XLE""","{""XLE"",26,237,29}","[{572555,115,3,86.41,255,255,115,57,255,""2025-07-24T09:30:04.216""}, {572641,0,200,86.425,255,32,255,57,255,""2025-07-24T09:30:04.247""}, … {577693,96,1,86.58,96,255,115,57,255,""2025-07-24T09:30:05.528""}]","[{10,3,7,0,68,86.45,86.39,0,""2025-07-24T09:30:04""}, {3,2,1,0,7,86.39,86.36,0,""2025-07-24T09:30:05""}, {14,4,7,0,68,86.36,86.34,0,""2025-07-24T09:30:06""}]","[{1799,86.425,86.37,86.39,26,86.37,86.425,""2025-07-24T09:30:04""}, {3471,86.37,86.34,86.37,54,86.34,86.37,""2025-07-24T09:30:05""}]"
2025-07-24 09:30:05,"""XLK""","{""XLK"",63,368,68}","[{571846,115,14,260.87,95,255,115,1,255,""2025-07-24T09:30:04.037""}, {571848,115,20,260.94,95,255,115,73,255,""2025-07-24T09:30:04.037""}, … {578191,115,1,260.7201,255,255,115,57,255,""2025-07-24T09:30:05.694""}]","[{2,3,68,0,68,260.94,260.74,0,""2025-07-24T09:30:04""}, {1,1,7,0,60,260.71,260.7,0,""2025-07-24T09:30:05""}, {1,2,60,0,7,260.78,260.66,0,""2025-07-24T09:30:06""}]","[{4272,260.84,260.71,260.77,64,260.71,260.84,""2025-07-24T09:30:04""}, {131,260.71,260.71,260.77,5,260.71,260.7

In [73]:
test_df

Event_Date_Time,Grade,Prog_Type,Bckgrnd,S1,S2,CT,CS,CB,Burst_Symbols,ms_of_day
datetime[μs],i64,i64,i64,i64,i64,i64,i64,i64,list[struct[4]],i64
2025-07-24 09:30:01,3,0,0,136,320,1,0,0,"[{""AALG"",1,204,61}, {""AAPL"",144,169,40}, … {""YINN"",10,241,44}]",34201000
2025-07-24 09:30:02,1,0,0,286,601,2,0,0,"[{""AAPB"",2,625,110}, {""AAPD"",1,360,101}, … {""ZOOZ"",47,48,39}]",34202000
2025-07-24 09:30:03,2,0,0,186,332,3,0,0,"[{""AAPB"",2,354,86}, {""AAPD"",2,410,98}, … {""YANG"",4,187,49}]",34203000
2025-07-24 09:30:04,2,0,0,156,343,4,0,0,"[{""AAPB"",1,274,53}, {""AAPD"",2,158,48}, … {""XYLD"",2,337,9}]",34204000
2025-07-24 09:30:05,3,0,0,86,152,5,0,0,"[{""AI"",281,608,137}, {""AIYY"",44,277,72}, … {""ZM"",243,427,131}]",34205000
…,…,…,…,…,…,…,…,…,…,…
2025-07-24 15:59:56,3,0,0,133,374,92,55,32,"[{""AAPU"",0,173,47}, {""AAPX"",4,294,101}, … {""XXRP"",0,236,66}]",57596000
2025-07-24 15:59:57,3,1,0,125,276,93,55,33,"[{""AAPD"",3,241,67}, {""AAPL"",429,1269,345}, … {""YINN"",12,165,35}]",57597000
2025-07-24 15:59:58,3,1,0,93,298,94,55,34,"[{""AAPL"",331,1036,352}, {""AAPU"",3,334,63}, … {""YINN"",5,190,42}]",57598000


In [83]:
def refactor_burst_df(burst_df: pl.DataFrame) -> pl.DataFrame:
    """
    For each unique Event_Date_Time, group the rows, combine the rest of the columns into a struct,
    and return a DataFrame with one row per unique Event_Date_Time and a list of Combined_Structs for each time.
    """
    if "Event_Date_Time" not in burst_df.columns:
        raise ValueError("Column 'Event_Date_Time' not found in DataFrame.")

    other_cols = [col for col in burst_df.columns if col != "Event_Date_Time"]
    struct_series = pl.struct([pl.col(col) for col in other_cols]).alias("Combined_Struct")

    # Add Combined_Struct column
    df_with_struct = burst_df.with_columns(struct_series)

    # For each unique Event_Date_Time, aggregate Combined_Structs into a list
    unique_dts = df_with_struct["Event_Date_Time"].unique()
    rows = []
    for dt in unique_dts:
        subset = df_with_struct.filter(pl.col("Event_Date_Time") == dt)
        combined_structs = subset["Combined_Struct"].to_list()
        rows.append({"Event_Date_Time": dt, "Combined_Struct": combined_structs})

    new_df = pl.DataFrame(rows)
    return new_df

final_df=refactor_burst_df(burst_df)
final_df

Event_Date_Time,Combined_Struct
datetime[μs],list[struct[5]]
2025-07-24 09:30:01,"[{""AALG"",{""AALG"",1,204,61},[{3303805,115,10,12.18,95,32,115,65,255,""2025-07-24T09:30:00.587""}, {3317786,62,675,12.13,62,32,45,1,255,""2025-07-24T09:30:00.822""}, {3317787,66,675,12.13,255,32,66,1,255,""2025-07-24T09:30:00.822""}],[{5,2,1,0,1,12.3,12.23,0,""2025-07-24T09:30:00""}, {5,1,1,0,1,12.2,12.14,0,""2025-07-24T09:30:01""}, {5,1,1,0,1,12.18,12.12,0,""2025-07-24T09:30:02""}],[{685,12.13,12.13,12.13,2,12.13,12.13,""2025-07-24T09:30:00""}, {0,0.0,0.0,12.13,0,0.0,0.0,""2025-07-24T09:30:01""}]}, {""AAPL"",{""AAPL"",144,169,40},[{3277779,115,17,213.82,95,32,115,60,255,""2025-07-24T09:30:00.016""}, {3277780,115,10,213.82,95,32,115,65,255,""2025-07-24T09:30:00.016""}, … {3405711,96,1,213.92,96,32,115,57,255,""2025-07-24T09:30:02""}],[{63,1,1,0,1,214.0,213.77,0,""2025-07-24T09:30:00""}, {1,1,65,0,7,213.8,213.74,0,""2025-07-24T09:30:01""}, {1,2,1,0,1,213.98,213.91,0,""2025-07-24T09:30:02""}],[{6681,213.98,213.77,213.84,169,213.77,213.9,""2025-07-24T09:30:00""}, {506509,213.98,213.69,213.91,844,213.98,213.79,""2025-07-24T09:30:01""}]}, … {""YINN"",{""YINN"",10,241,44},[{500908,62,26507,48.48,62,255,255,7,255,""2025-07-24T09:30:00.069""}, {500909,66,26507,48.48,255,255,66,7,255,""2025-07-24T09:30:00.069""}, … {558164,115,20,48.4,255,255,115,7,255,""2025-07-24T09:30:01.927""}],[{15,47,7,0,7,48.48,48.32,0,""2025-07-24T09:30:00""}, {5,1,7,0,7,48.48,48.42,0,""2025-07-24T09:30:01""}, {1,1,7,0,7,48.41,48.39,0,""2025-07-24T09:30:02""}],[{27682,48.48,48.45,48.47,21,48.45,48.48,""2025-07-24T09:30:00""}, {1317,48.42,48.41,48.47,25,48.41,48.42,""2025-07-24T09:30:01""}]}]"
2025-07-24 09:30:02,"[{""AAPB"",{""AAPB"",2,625,110},[{3349673,115,2,21.72,255,32,115,57,255,""2025-07-24T09:30:01.297""}, {3431073,96,1,21.7,96,32,115,57,255,""2025-07-24T09:30:02.469""}, … {3444079,95,100,21.8,95,32,255,60,255,""2025-07-24T09:30:02.78""}],[{12,12,65,0,65,21.73,21.71,0,""2025-07-24T09:30:01""}, {12,12,65,0,65,21.77,21.74,0,""2025-07-24T09:30:02""}, {12,12,65,0,60,21.83,21.8,0,""2025-07-24T09:30:03""}],[{2,0.0,0.0,0.0,1,0.0,0.0,""2025-07-24T09:30:01""}, {201,21.8,21.8,21.8,3,21.8,21.8,""2025-07-24T09:30:02""}]}, {""AAPD"",{""AAPD"",1,360,101},[{3408636,115,1,16.86,95,32,115,43,255,""2025-07-24T09:30:02.047""}, {3422765,96,1,16.87,96,32,115,57,255,""2025-07-24T09:30:02.306""}, … {3440079,96,1,16.8889,96,32,115,57,255,""2025-07-24T09:30:02.671""}],[{33,32,65,0,43,16.89,16.88,0,""2025-07-24T09:30:01""}, {9,41,1,0,1,16.87,16.86,0,""2025-07-24T09:30:02""}, {32,32,65,0,1,16.85,16.84,0,""2025-07-24T09:30:03""}],[{0,0.0,0.0,0.0,0,0.0,0.0,""2025-07-24T09:30:01""}, {8,0.0,0.0,0.0,4,0.0,0.0,""2025-07-24T09:30:02""}]}, … {""ZOOZ"",{""ZOOZ"",47,48,39},[{3331238,115,3,2.82,255,32,115,68,255,""2025-07-24T09:30:01.034""}, {3348954,115,1,2.815,255,32,115,65,255,""2025-07-24T09:30:01.287""}, … {3449971,115,53,2.815,255,32,115,57,255,""2025-07-24T09:30:02.967""}],[{1,1,1,0,1,2.83,2.81,0,""2025-07-24T09:30:01""}, {1,9,1,0,1,2.83,2.8,0,""2025-07-24T09:30:02""}, {1,9,1,0,1,2.83,2.8,0,""2025-07-24T09:30:03""}],[{8,0.0,0.0,0.0,6,0.0,0.0,""2025-07-24T09:30:01""}, {1489,2.815,2.8001,2.8,16,2.815,2.8138,""2025-07-24T09:30:02""}]}]"
2025-07-24 09:30:03,"[{""AAPB"",{""AAPB"",2,354,86},[{3431073,96,1,21.7,96,32,115,57,255,""2025-07-24T09:30:02.469""}, {3444076,0,100,21.8,255,32,255,1,255,""2025-07-24T09:30:02.78""}, … {3474176,0,100,21.81,255,32,255,57,255,""2025-07-24T09:30:03.842""}],[{12,12,65,0,65,21.77,21.74,0,""2025-07-24T09:30:02""}, {12,12,65,0,60,21.83,21.8,0,""2025-07-24T09:30:03""}, {12,12,65,0,65,21.8,21.77,0,""2025-07-24T09:30:04""}],[{201,21.8,21.8,21.8,3,21.8,21.8,""2025-07-24T09:30:02""}, {100,21.81,21.81,21.8,1,21.81,21.81,""2025-07-24T09:30:03""}]}, {""AAPD"",{""AAPD"",2,410,98},[{3408636,115,1,16.86,95,32,115,43,255,""2025-07-24T09:30:02.047""}, {3422765,96,1,16.87,96,32,115,57,255,""2025-07-24T09:30:02.306""}, … {3468183,95,794,16.85,95,32,255,1,255,""2025-07-24T09:30:03.561""}],[{9,

In [12]:
def extract_date_strings(merged_df: pl.DataFrame) -> tuple[str, str]:
    """
    Extracts the date from the first Event_Date_Time in merged_df and returns a tuple:
    ( 'YYYY/%m/', 'YYYYMMDD' )
    """
    # Get the first Event_Date_Time value
    first_dt = merged_df["Event_Date_Time"][0]
    # If it's a string, parse to datetime
    if isinstance(first_dt, str):
        first_dt = datetime.datetime.fromisoformat(first_dt)
    # Format as 'YYYY/%m/' and 'YYYYMMDD'
    return first_dt.strftime('%Y/%m/'), first_dt.strftime('%Y%m%d')

# Example usage:
today_str="20250724"
today_str1="2025/07/"
# today_str1, today_str = extract_date_strings(merged_df)
print(f"Extracted date strings: {today_str1}, {today_str}")

Extracted date strings: 2025/07/, 20250724


In [92]:
merged_df = test_df.head(5).join(final_df, on="Event_Date_Time", how="left")

burst_detail_parquet_path = f"./Processed_Data/{today_str1}{today_str}_burst_detail.parquet"

# Save merged_df to parquet at burst_detail_parquet_path
merged_df.write_parquet(burst_detail_parquet_path)
print(f"Saved merged_df to {burst_detail_parquet_path}")

merged_df

Saved merged_df to ./Processed_Data/2025/07/20250724_burst_detail.parquet


Event_Date_Time,Grade,Prog_Type,Bckgrnd,S1,S2,CT,CS,CB,Burst_Symbols,ms_of_day,Combined_Struct
datetime[μs],i64,i64,i64,i64,i64,i64,i64,i64,list[struct[4]],i64,list[struct[5]]
2025-07-24 09:30:01,3,0,0,136,320,1,0,0,"[{""AALG"",1,204,61}, {""AAPL"",144,169,40}, … {""YINN"",10,241,44}]",34201000,"[{""AALG"",{""AALG"",1,204,61},[{3303805,115,10,12.18,95,32,115,65,255,""2025-07-24T09:30:00.587""}, {3317786,62,675,12.13,62,32,45,1,255,""2025-07-24T09:30:00.822""}, {3317787,66,675,12.13,255,32,66,1,255,""2025-07-24T09:30:00.822""}],[{5,2,1,0,1,12.3,12.23,0,""2025-07-24T09:30:00""}, {5,1,1,0,1,12.2,12.14,0,""2025-07-24T09:30:01""}, {5,1,1,0,1,12.18,12.12,0,""2025-07-24T09:30:02""}],[{685,12.13,12.13,12.13,2,12.13,12.13,""2025-07-24T09:30:00""}, {0,0.0,0.0,12.13,0,0.0,0.0,""2025-07-24T09:30:01""}]}, {""AAPL"",{""AAPL"",144,169,40},[{3277779,115,17,213.82,95,32,115,60,255,""2025-07-24T09:30:00.016""}, {3277780,115,10,213.82,95,32,115,65,255,""2025-07-24T09:30:00.016""}, … {3405711,96,1,213.92,96,32,115,57,255,""2025-07-24T09:30:02""}],[{63,1,1,0,1,214.0,213.77,0,""2025-07-24T09:30:00""}, {1,1,65,0,7,213.8,213.74,0,""2025-07-24T09:30:01""}, {1,2,1,0,1,213.98,213.91,0,""2025-07-24T09:30:02""}],[{6681,213.98,213.77,213.84,169,213.77,213.9,""2025-07-24T09:30:00""}, {506509,213.98,213.69,213.91,844,213.98,213.79,""2025-07-24T09:30:01""}]}, … {""YINN"",{""YINN"",10,241,44},[{500908,62,26507,48.48,62,255,255,7,255,""2025-07-24T09:30:00.069""}, {500909,66,26507,48.48,255,255,66,7,255,""2025-07-24T09:30:00.069""}, … {558164,115,20,48.4,255,255,115,7,255,""2025-07-24T09:30:01.927""}],[{15,47,7,0,7,48.48,48.32,0,""2025-07-24T09:30:00""}, {5,1,7,0,7,48.48,48.42,0,""2025-07-24T09:30:01""}, {1,1,7,0,7,48.41,48.39,0,""2025-07-24T09:30:02""}],[{27682,48.48,48.45,48.47,21,48.45,48.48,""2025-07-24T09:30:00""}, {1317,48.42,48.41,48.47,25,48.41,48.42,""2025-07-24T09:30:01""}]}]"
2025-07-24 09:30:02,1,0,0,286,601,2,0,0,"[{""AAPB"",2,625,110}, {""AAPD"",1,360,101}, … {""ZOOZ"",47,48,39}]",34202000,"[{""AAPB"",{""AAPB"",2,625,110},[{3349673,115,2,21.72,255,32,115,57,255,""2025-07-24T09:30:01.297""}, {3431073,96,1,21.7,96,32,115,57,255,""2025-07-24T09:30:02.469""}, … {3444079,95,100,21.8,95,32,255,60,255,""2025-07-24T09:30:02.78""}],[{12,12,65,0,65,21.73,21.71,0,""2025-07-24T09:30:01""}, {12,12,65,0,65,21.77,21.74,0,""2025-07-24T09:30:02""}, {12,12,65,0,60,21.83,21.8,0,""2025-07-24T09:30:03""}],[{2,0.0,0.0,0.0,1,0.0,0.0,""2025-07-24T09:30:01""}, {201,21.8,21.8,21.8,3,21.8,21.8,""2025-07-24T09:30:02""}]}, {""AAPD"",{""AAPD"",1,360,101},[{3408636,115,1,16.86,95,32,115,43,255,""2025-07-24T09:30:02.047""}, {3422765,96,1,16.87,96,32,115,57,255,""2025-07-24T09:30:02.306""}, … {3440079,96,1,16.8889,96,32,115,57,255,""2025-07-24T09:30:02.671""}],[{33,32,65,0,43,16.89,16.88,0,""2025-07-24T09:30:01""}, {9,41,1,0,1,16.87,16.86,0,""2025-07-24T09:30:02""}, {32,32,65,0,1,16.85,16.84,0,""2025-07-24T09:30:03""}],[{0,0.0,0.0,0.0,0,0.0,0.0,""2025-07-24T09:30:01""}, {8,0.0,0.0,0.0,4,0.0,0.0,""2025-07-24T09:30:02""}]}, … {""ZOOZ"",{""ZOOZ"",47,48,39},[{3331238,115,3,2.82,255,32,115,68,255,""2025-07-24T09:30:01.034""}, {3348954,115,1,2.815,255,32,115,65,255,""2025-07-24T09:30:01.287""}, … {3449971,115,53,2.815,255,32,115,57,255,""2025-07-24T09:30:02.967""}],[{1,1,1,0,1,2.83,2.81,0,""2025-07-24T09:30:01""}, {1,9,1,0,1,2.83,2.8,0,""2025-07-24T09:30:02""}, {1,9,1,0,1,2.83,2.8,0,""2025-07-24T09:30:03""}],[{8,0.0,0.0,0.0,6,0.0,0.0,""2025-07-24T09:30:01""}, {1489,2.815,2.8001,2.8,16,2.815,2.8138,""2025-07-24T09:30:02""}]}]"
2025-07-24 09:30:03,2,0,0,186,332,3,0,0,"[{""AAPB"",2,354,86}, {""AAPD"",2,410,98}, … {""YANG"",4,187,49}]",34203000,"[{""AAPB"",{""AAPB"",2,354,86},[{3431073,96,1,21.7,96,32,115,57,255,""2025-07-24T09:30:02.469""}, {3444076,0,100,21.8,255,32,255,1,255,""2025-07-24T09:30:02.78""}, … {3474176,0,100,21.81,255,32,255,57,255,""2025-07-24T09:30:03.842""}],[{12,12,65,0,65,21.77,21.74,0,""2025-07-24T09:30:02""}, {12,12,65,0,60,21.83,21.8,0,""2025-07-24T09:30:03""}, {12,12

In [13]:
def expand_combined_struct(df: pl.DataFrame) -> pl.DataFrame:
    """
    Expands the 'Combined_Struct' column of a DataFrame (created by refactor_burst_df)
    back into its original columns, one row per struct element.
    """
    # Explode the Combined_Struct column so each struct is a row
    exploded = df.explode("Combined_Struct")
    # Expand the struct into columns
    expanded = exploded.with_columns(
        pl.col("Combined_Struct").struct.unnest()
    ).drop("Combined_Struct")
    return expanded

#open merged data parquet file
recovered_merged_df = pl.read_parquet(burst_detail_parquet_path)

# Example usage:
original_df = expand_combined_struct(recovered_merged_df)
original_df

Event_Date_Time,Grade,Prog_Type,Bckgrnd,S1,S2,CT,CS,CB,Burst_Symbols,ms_of_day,Symbol,Burst_Info,Trade_History,Quote_History,OHLC
datetime[μs],i64,i64,i64,i64,i64,i64,i64,i64,list[struct[4]],i64,str,struct[4],list[struct[10]],list[struct[9]],list[struct[8]]
2025-07-24 09:30:01,3,0,0,136,320,1,0,0,"[{""AALG"",1,204,61}, {""AAPL"",144,169,40}, … {""YINN"",10,241,44}]",34201000,"""AALG""","{""AALG"",1,204,61}","[{3303805,115,10,12.18,95,32,115,65,255,""2025-07-24T09:30:00.587""}, {3317786,62,675,12.13,62,32,45,1,255,""2025-07-24T09:30:00.822""}, {3317787,66,675,12.13,255,32,66,1,255,""2025-07-24T09:30:00.822""}]","[{5,2,1,0,1,12.3,12.23,0,""2025-07-24T09:30:00""}, {5,1,1,0,1,12.2,12.14,0,""2025-07-24T09:30:01""}, {5,1,1,0,1,12.18,12.12,0,""2025-07-24T09:30:02""}]","[{685,12.13,12.13,12.13,2,12.13,12.13,""2025-07-24T09:30:00""}, {0,0.0,0.0,12.13,0,0.0,0.0,""2025-07-24T09:30:01""}]"
2025-07-24 09:30:01,3,0,0,136,320,1,0,0,"[{""AALG"",1,204,61}, {""AAPL"",144,169,40}, … {""YINN"",10,241,44}]",34201000,"""AAPL""","{""AAPL"",144,169,40}","[{3277779,115,17,213.82,95,32,115,60,255,""2025-07-24T09:30:00.016""}, {3277780,115,10,213.82,95,32,115,65,255,""2025-07-24T09:30:00.016""}, … {3405711,96,1,213.92,96,32,115,57,255,""2025-07-24T09:30:02""}]","[{63,1,1,0,1,214.0,213.77,0,""2025-07-24T09:30:00""}, {1,1,65,0,7,213.8,213.74,0,""2025-07-24T09:30:01""}, {1,2,1,0,1,213.98,213.91,0,""2025-07-24T09:30:02""}]","[{6681,213.98,213.77,213.84,169,213.77,213.9,""2025-07-24T09:30:00""}, {506509,213.98,213.69,213.91,844,213.98,213.79,""2025-07-24T09:30:01""}]"
2025-07-24 09:30:01,3,0,0,136,320,1,0,0,"[{""AALG"",1,204,61}, {""AAPL"",144,169,40}, … {""YINN"",10,241,44}]",34201000,"""AAPU""","{""AAPU"",4,192,41}","[{3295621,1,1,23.64,255,32,115,1,1,""2025-07-24T09:30:00.418""}, {3295622,1,29,23.64,255,32,115,1,1,""2025-07-24T09:30:00.418""}, … {3395101,115,1,23.67,255,32,115,58,255,""2025-07-24T09:30:01.857""}]","[{13,13,1,0,1,23.7,23.62,0,""2025-07-24T09:30:00""}, {11,11,65,0,65,23.65,23.63,0,""2025-07-24T09:30:01""}, {11,12,65,0,11,23.69,23.66,0,""2025-07-24T09:30:02""}]","[{36,0.0,0.0,0.0,4,0.0,0.0,""2025-07-24T09:30:00""}, {38113,23.67,23.63,23.65,33,23.67,23.66,""2025-07-24T09:30:01""}]"
2025-07-24 09:30:01,3,0,0,136,320,1,0,0,"[{""AALG"",1,204,61}, {""AAPL"",144,169,40}, … {""YINN"",10,241,44}]",34201000,"""ACMR""","{""ACMR"",21,166,25}","[{3281324,62,7935,30.26,62,32,45,1,255,""2025-07-24T09:30:00.094""}, {3281349,66,7935,30.26,255,32,66,1,255,""2025-07-24T09:30:00.094""}, … {3404322,115,1,30.26,255,32,115,57,255,""2025-07-24T09:30:01.975""}]","[{1,1,65,0,7,30.4,29.95,0,""2025-07-24T09:30:00""}, {1,4,1,0,7,30.29,30.0,0,""2025-07-24T09:30:01""}, {1,1,1,0,1,30.29,30.0,0,""2025-07-24T09:30:02""}]","[{8307,30.26,30.125,30.25,25,30.125,30.26,""2025-07-24T09:30:00""}, {95,0.0,0.0,30.25,8,0.0,0.0,""2025-07-24T09:30:01""}]"
2025-07-24 09:30:01,3,0,0,136,320,1,0,0,"[{""AALG"",1,204,61}, {""AAPL"",144,169,40}, … {""YINN"",10,241,44}]",34201000,"""ACWI""","{""ACWI"",15,151,22}","[{3295182,1,5,131.79,95,32,115,1,1,""2025-07-24T09:30:00.41""}, {3295183,1,123,131.8,95,32,255,1,1,""2025-07-24T09:30:00.41""}, … {3395187,96,1,131.8,96,32,115,57,255,""2025-07-24T09:30:01.858""}]","[{11,11,1,0,7,131.8,131.64,0,""2025-07-24T09:30:00""}, {15,7,1,0,1,131.8,131.66,0,""2025-07-24T09:30:01""}, {15,4,1,0,7,131.8,131.66,0,""2025-07-24T09:30:02""}]","[{9064,131.8,131.8,131.8,16,131.8,131.8,""2025-07-24T09:30:00""}, {511,131.73,131.73,131.79,20,131.73,131.73,""2025-07-24T09:30:01""}]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-07-24 09:30:05,3,0,0,86,152,5,0,0,"[{""AI"",281,608,137}, {""AIYY"",44,277,72}, … {""ZM"",243,427,131}]",34205000,"""XLE""","{""XLE"",26,237,29}","[{572555,115,3,86.41,255,255,115,57,255,""2025-07-24T09:30:04.216""}, {572641,0,200,86.425,255,32,255,57,255,""2025-07-24T09:30:04.247""}, … {577693,96,1,86.58,96,255,115,57,255,""2025-07-24T09:30:05.528""}]","[{10,3,7,0,68,86.45,86.39,0,""2025-07-24T09:30:04""}, {3,2,1,0,7,86.39,86.36,0,""2025-07-24T09:30:05""}, 

In [89]:
recovered_merged_df

Event_Date_Time,Grade,Prog_Type,Bckgrnd,S1,S2,CT,CS,CB,Burst_Symbols,ms_of_day,Combined_Struct
datetime[μs],i64,i64,i64,i64,i64,i64,i64,i64,list[struct[4]],i64,list[struct[5]]
2025-07-24 09:30:01,3,0,0,136,320,1,0,0,"[{""AALG"",1,204,61}, {""AAPL"",144,169,40}, … {""YINN"",10,241,44}]",34201000,"[{""AALG"",{""AALG"",1,204,61},[{3303805,115,10,12.18,95,32,115,65,255,""2025-07-24T09:30:00.587""}, {3317786,62,675,12.13,62,32,45,1,255,""2025-07-24T09:30:00.822""}, {3317787,66,675,12.13,255,32,66,1,255,""2025-07-24T09:30:00.822""}],[{5,2,1,0,1,12.3,12.23,0,""2025-07-24T09:30:00""}, {5,1,1,0,1,12.2,12.14,0,""2025-07-24T09:30:01""}, {5,1,1,0,1,12.18,12.12,0,""2025-07-24T09:30:02""}],[{685,12.13,12.13,12.13,2,12.13,12.13,""2025-07-24T09:30:00""}, {0,0.0,0.0,12.13,0,0.0,0.0,""2025-07-24T09:30:01""}]}, {""AAPL"",{""AAPL"",144,169,40},[{3277779,115,17,213.82,95,32,115,60,255,""2025-07-24T09:30:00.016""}, {3277780,115,10,213.82,95,32,115,65,255,""2025-07-24T09:30:00.016""}, … {3405711,96,1,213.92,96,32,115,57,255,""2025-07-24T09:30:02""}],[{63,1,1,0,1,214.0,213.77,0,""2025-07-24T09:30:00""}, {1,1,65,0,7,213.8,213.74,0,""2025-07-24T09:30:01""}, {1,2,1,0,1,213.98,213.91,0,""2025-07-24T09:30:02""}],[{6681,213.98,213.77,213.84,169,213.77,213.9,""2025-07-24T09:30:00""}, {506509,213.98,213.69,213.91,844,213.98,213.79,""2025-07-24T09:30:01""}]}, … {""YINN"",{""YINN"",10,241,44},[{500908,62,26507,48.48,62,255,255,7,255,""2025-07-24T09:30:00.069""}, {500909,66,26507,48.48,255,255,66,7,255,""2025-07-24T09:30:00.069""}, … {558164,115,20,48.4,255,255,115,7,255,""2025-07-24T09:30:01.927""}],[{15,47,7,0,7,48.48,48.32,0,""2025-07-24T09:30:00""}, {5,1,7,0,7,48.48,48.42,0,""2025-07-24T09:30:01""}, {1,1,7,0,7,48.41,48.39,0,""2025-07-24T09:30:02""}],[{27682,48.48,48.45,48.47,21,48.45,48.48,""2025-07-24T09:30:00""}, {1317,48.42,48.41,48.47,25,48.41,48.42,""2025-07-24T09:30:01""}]}]"
2025-07-24 09:30:02,1,0,0,286,601,2,0,0,"[{""AAPB"",2,625,110}, {""AAPD"",1,360,101}, … {""ZOOZ"",47,48,39}]",34202000,"[{""AAPB"",{""AAPB"",2,625,110},[{3349673,115,2,21.72,255,32,115,57,255,""2025-07-24T09:30:01.297""}, {3431073,96,1,21.7,96,32,115,57,255,""2025-07-24T09:30:02.469""}, … {3444079,95,100,21.8,95,32,255,60,255,""2025-07-24T09:30:02.78""}],[{12,12,65,0,65,21.73,21.71,0,""2025-07-24T09:30:01""}, {12,12,65,0,65,21.77,21.74,0,""2025-07-24T09:30:02""}, {12,12,65,0,60,21.83,21.8,0,""2025-07-24T09:30:03""}],[{2,0.0,0.0,0.0,1,0.0,0.0,""2025-07-24T09:30:01""}, {201,21.8,21.8,21.8,3,21.8,21.8,""2025-07-24T09:30:02""}]}, {""AAPD"",{""AAPD"",1,360,101},[{3408636,115,1,16.86,95,32,115,43,255,""2025-07-24T09:30:02.047""}, {3422765,96,1,16.87,96,32,115,57,255,""2025-07-24T09:30:02.306""}, … {3440079,96,1,16.8889,96,32,115,57,255,""2025-07-24T09:30:02.671""}],[{33,32,65,0,43,16.89,16.88,0,""2025-07-24T09:30:01""}, {9,41,1,0,1,16.87,16.86,0,""2025-07-24T09:30:02""}, {32,32,65,0,1,16.85,16.84,0,""2025-07-24T09:30:03""}],[{0,0.0,0.0,0.0,0,0.0,0.0,""2025-07-24T09:30:01""}, {8,0.0,0.0,0.0,4,0.0,0.0,""2025-07-24T09:30:02""}]}, … {""ZOOZ"",{""ZOOZ"",47,48,39},[{3331238,115,3,2.82,255,32,115,68,255,""2025-07-24T09:30:01.034""}, {3348954,115,1,2.815,255,32,115,65,255,""2025-07-24T09:30:01.287""}, … {3449971,115,53,2.815,255,32,115,57,255,""2025-07-24T09:30:02.967""}],[{1,1,1,0,1,2.83,2.81,0,""2025-07-24T09:30:01""}, {1,9,1,0,1,2.83,2.8,0,""2025-07-24T09:30:02""}, {1,9,1,0,1,2.83,2.8,0,""2025-07-24T09:30:03""}],[{8,0.0,0.0,0.0,6,0.0,0.0,""2025-07-24T09:30:01""}, {1489,2.815,2.8001,2.8,16,2.815,2.8138,""2025-07-24T09:30:02""}]}]"
2025-07-24 09:30:03,2,0,0,186,332,3,0,0,"[{""AAPB"",2,354,86}, {""AAPD"",2,410,98}, … {""YANG"",4,187,49}]",34203000,"[{""AAPB"",{""AAPB"",2,354,86},[{3431073,96,1,21.7,96,32,115,57,255,""2025-07-24T09:30:02.469""}, {3444076,0,100,21.8,255,32,255,1,255,""2025-07-24T09:30:02.78""}, … {3474176,0,100,21.81,255,32,255,57,255,""2025-07-24T09:30:03.842""}],[{12,12,65,0,65,21.77,21.74,0,""2025-07-24T09:30:02""}, {12,12,65,0,60,21.83,21.8,0,""2025-07-24T09:30:03""}, {12,12

In [ ]:
# Filter original_df for EEvent_Date_Time
filtered_df = original_df[original_df["Event_Date_Time"]==] 

# Development cell to incrementall achieve function ... temporary only

In [ ]:
BASE_URL = "http://localhost:25504/v3/stock/history/trade"
INTERVAL = 10  # ms
FORMAT = "ndjson"

results = []

for row in test_df.iter_rows(named=True):
    event_dt = row["Event_Date_Time"]
    burst_symbols = row["Burst_Symbols"]
    if not burst_symbols:
        continue
    break
print(f'Processing Event Date Time: {event_dt}')
print(f'Burst Symbols: {len(burst_symbols)}, {burst_symbols}')

# Prepare date
date_str = event_dt.strftime("%Y%m%d")
ms_of_day = (event_dt.hour * 3600 + event_dt.minute * 60 + event_dt.second) * 1000 + int(event_dt.microsecond / 1000)
# Calculate start and end times in ms
start_ms = max(ms_of_day - 1000, 0)
end_ms = ms_of_day + 1000

# Convert ms_of_day to HH.MM.SS.mmm format
def ms_to_time_str(ms):
    hours = ms // 3600000
    ms %= 3600000
    minutes = ms // 60000
    ms %= 60000
    seconds = ms // 1000
    milliseconds = ms % 1000
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}.{milliseconds:03d}"

start_time_str = ms_to_time_str(start_ms)
end_time_str = ms_to_time_str(end_ms)

print(f'Start Time: {start_time_str}, End Time: {end_time_str}')

for symbol_info in burst_symbols:
    symbol = symbol_info.get("Symbol")
    print(f'Processing Symbol: {symbol}')
    if not symbol:
        continue

    params = {
        "symbol": symbol,
        "date": date_str,
        "start_time": start_time_str,
        "end_time": end_time_str,
        "format": FORMAT
            }
    
    # print(f'Params: {params}')



Processing Event Date Time: 2025-07-24 09:30:01
Burst Symbols: 319, [{'Symbol': 'AALG', 'NumTrades': 1, 'NumQuotes': 204, 'BBO': 61}, {'Symbol': 'AAPL', 'NumTrades': 144, 'NumQuotes': 169, 'BBO': 40}, {'Symbol': 'AAPU', 'NumTrades': 4, 'NumQuotes': 192, 'BBO': 41}, {'Symbol': 'ACMR', 'NumTrades': 21, 'NumQuotes': 166, 'BBO': 25}, {'Symbol': 'ACWI', 'NumTrades': 15, 'NumQuotes': 151, 'BBO': 22}, {'Symbol': 'ACWX', 'NumTrades': 3, 'NumQuotes': 163, 'BBO': 20}, {'Symbol': 'ADT', 'NumTrades': 26, 'NumQuotes': 228, 'BBO': 49}, {'Symbol': 'AG', 'NumTrades': 124, 'NumQuotes': 980, 'BBO': 255}, {'Symbol': 'AGH', 'NumTrades': 27, 'NumQuotes': 200, 'BBO': 67}, {'Symbol': 'AGNC', 'NumTrades': 42, 'NumQuotes': 204, 'BBO': 65}, {'Symbol': 'AIRR', 'NumTrades': 2, 'NumQuotes': 181, 'BBO': 31}, {'Symbol': 'ALV', 'NumTrades': 13, 'NumQuotes': 189, 'BBO': 51}, {'Symbol': 'AMD', 'NumTrades': 529, 'NumQuotes': 775, 'BBO': 268}, {'Symbol': 'AMDD', 'NumTrades': 1, 'NumQuotes': 174, 'BBO': 19}, {'Symbol': 'A

In [52]:

symbol="AAPU"

params = {
    "symbol": symbol,
    "date": date_str,
    "start_time": start_time_str,
    "end_time": end_time_str,
    "format": FORMAT
        }
try:
    response = httpx.get(BASE_URL, params=params, timeout=10)
    response.raise_for_status()
    # Parse NDJSON response (one JSON object per line)
    trades = [json.loads(line) for line in response.text.strip().split('\n') if line.strip()]
except Exception as e:
    trades = []

# convert trades to polars dataframe
trades_df = pl.DataFrame(trades)

print(f'Params: {params}')

print(f'Fetched {len(trades)} trades for symbol {symbol}')

# # Combine the burst symbol info and trade data
# results.append({
#     "Event_Date_Time": event_dt,
#     "Symbol": symbol,
#     "Burst_Info": symbol_info,
#     "Trade_History": trades
# })
trades_df

Params: {'symbol': 'AAPU', 'date': '20250724', 'start_time': '09:30:00.000', 'end_time': '09:30:02.000', 'format': 'ndjson'}
Fetched 39 trades for symbol AAPU


sequence,condition,size,price,ext_condition2,ext_condition1,ext_condition4,exchange,ext_condition3,timestamp
i64,i64,i64,f64,i64,i64,i64,i64,i64,str
3295621,1,1,23.64,255,32,115,1,1,"""2025-07-24T09:30:00.418"""
3295622,1,29,23.64,255,32,115,1,1,"""2025-07-24T09:30:00.418"""
3296927,115,1,23.65,255,32,115,57,255,"""2025-07-24T09:30:00.447"""
3296978,115,5,23.64,255,32,115,57,255,"""2025-07-24T09:30:00.449"""
3335214,1,100,23.64,255,32,255,1,1,"""2025-07-24T09:30:01.083"""
…,…,…,…,…,…,…,…,…,…
3346188,0,200,23.65,255,32,255,1,255,"""2025-07-24T09:30:01.246"""
3350752,0,200,23.64,255,32,255,8,255,"""2025-07-24T09:30:01.311"""
3379722,0,100,23.67,255,32,255,1,255,"""2025-07-24T09:30:01.687"""


In [46]:
response.text

'{"sequence":3277779,"condition":115,"size":17,"price":213.820000,"ext_condition2":95,"ext_condition1":32,"ext_condition4":115,"exchange":60,"ext_condition3":255,"timestamp":"2025-07-24T09:30:00.016"}\n{"sequence":3277780,"condition":115,"size":10,"price":213.820000,"ext_condition2":95,"ext_condition1":32,"ext_condition4":115,"exchange":65,"ext_condition3":255,"timestamp":"2025-07-24T09:30:00.016"}\n{"sequence":3277800,"condition":115,"size":1,"price":213.820000,"ext_condition2":95,"ext_condition1":32,"ext_condition4":115,"exchange":65,"ext_condition3":255,"timestamp":"2025-07-24T09:30:00.016"}\n{"sequence":3278283,"condition":115,"size":1,"price":214.000000,"ext_condition2":255,"ext_condition1":32,"ext_condition4":115,"exchange":57,"ext_condition3":255,"timestamp":"2025-07-24T09:30:00.029"}\n{"sequence":3278371,"condition":115,"size":1,"price":213.820000,"ext_condition2":255,"ext_condition1":32,"ext_condition4":115,"exchange":60,"ext_condition3":255,"timestamp":"2025-07-24T09:30:00.03